In [2]:
# ==============================================================
# 📂 Google Drive + Package Setup (Module 5 - Export)
# ==============================================================

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set project/data paths
PROJECT_PATH = "/content/drive/MyDrive/el-roi"
DATA_PATH = os.path.join(PROJECT_PATH, "data")
REPORTS_PATH = os.path.join(PROJECT_PATH, "reports")
os.makedirs(REPORTS_PATH, exist_ok=True)

print(f"✅ Reports will be saved to: {REPORTS_PATH}")

# ==============================================================
# 📦 Install Required Packages
# ==============================================================
!pip install --quiet markdown2


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Reports will be saved to: /content/drive/MyDrive/el-roi/reports


In [3]:
# ==============================================================
# MODULE 5: Export Clustered Reports as Markdown
# ==============================================================

import pandas as pd
import markdown2
from datetime import datetime

# Load clustered articles
cluster_input_path = os.path.join(DATA_PATH, "articles_clustered.jsonl")
if not os.path.exists(cluster_input_path):
    raise FileNotFoundError("❌ Clustered article file not found. Run Module 4 first.")

df = pd.read_json(cluster_input_path, lines=True)

# --------------------------------------------------------------
# 📦 Report Generator
# --------------------------------------------------------------
def generate_markdown_report(df, cluster_id):
    """
    Generate a Markdown report string for a specific cluster.
    """
    cluster_df = df[df["cluster_id"] == cluster_id]
    if cluster_df.empty:
        return ""

    report_lines = [
        f"# 🧠 EL ROI Cluster Report: Cluster {cluster_id}",
        f"**Document Count:** {len(cluster_df)}",
        f"**Generated on:** {datetime.now().strftime('%Y-%m-%d %H:%M')}",
        "---"
    ]

    for idx, row in cluster_df.iterrows():
        title = row.get("title", "Untitled")
        link = row.get("link", "")
        source = row.get("source", "")
        translated_summary = row.get("translated_summary", "")
        entities = row.get("entities", [])

        entity_str = ", ".join([f"`{e['text']} ({e['label']})`" for e in entities])

        report_lines += [
            f"## 📄 {title}",
            f"**Source:** {source}  |  **Link:** [View Article]({link})",
            f"**Entities:** {entity_str}",
            "",
            translated_summary,
            "---"
        ]

    return "\n".join(report_lines)

# --------------------------------------------------------------
# 💾 Export All Clusters to Markdown
# --------------------------------------------------------------
clusters = sorted(df["cluster_id"].unique())

for cluster_id in clusters:
    if cluster_id == -1:
        continue  # Skip noise

    md_text = generate_markdown_report(df, cluster_id)
    if not md_text:
        continue

    md_filename = f"cluster_{cluster_id}.md"
    md_path = os.path.join(REPORTS_PATH, md_filename)

    with open(md_path, "w", encoding="utf-8") as f:
        f.write(md_text)

    print(f"✅ Exported: {md_filename}")

print("✅ All cluster reports generated in Markdown.")


✅ Exported: cluster_0.md
✅ Exported: cluster_1.md
✅ All cluster reports generated in Markdown.
